### Dependencias del entorno
A continuación se listan todas las librerías instaladas asi como sus respectivas versiones




In [ ]:
pip list

Package                   Version
------------------------- -----------
anyio                     4.12.1
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.4.0
asttokens                 3.0.1
async-lru                 2.1.0
attrs                     25.4.0
babel                     2.17.0
beautifulsoup4            4.14.3
bleach                    6.3.0
certifi                   2026.1.4
cffi                      2.0.0
charset-normalizer        3.4.4
colorama                  0.4.6
comm                      0.2.3
debugpy                   1.8.19
decorator                 5.2.1
defusedxml                0.7.1
executing                 2.2.1
fastjsonschema            2.21.2
feedparser                6.0.12
filelock                  3.20.3
fqdn                      1.5.1
fsspec                    2026.1.0
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
huggingface-hub           0.36.0
idna         

In [ ]:
#pip install requests feedparser


### Definición de lista blanca de dominios (GDELT / RSS)

En esta celda se define `WHITELIST_DOMAINS`, un conjunto de dominios considerados **fuentes financieras y periodísticas fiables**

Esta lista se utiliza posteriormente para:

- Filtrar los resultados devueltos por GDELT y otras fuentes RSS.
- Asegurar que solo se tienen en cuenta medios de comunicación relevantes para el análisis financiero del presente trabajo.
- Reducir ruido y posibles sesgos derivados de fuentes de baja calidad.
- Consulta a sitios no confiables.

Esta decisión de diseño forma parte del control de calidad de los datos de entrada al modelo de sentimiento.


In [ ]:
#whitelists para GDELT
WHITELIST_DOMAINS = {
    "reuters.com",
    "bloomberg.com",
    "wsj.com",
    "ft.com",
    "cnbc.com",
    "finance.yahoo.com",
    "marketwatch.com",
    "investing.com",
    "seekingalpha.com",
    "fool.com",
    "morningstar.com",
    "thestreet.com",
    "barrons.com",
    "businessinsider.com",
    "apnews.com",
    "abcnews.go.com",
    "nytimes.com",
    "washingtonpost.com",
}

def domain_allowed(domain: str, whitelist: set[str]) -> bool:
    if not domain:
        return False
    d = domain.lower().strip()
    return any(d == w or d.endswith("." + w) for w in whitelist)

# Declaración de los helpers de preprocesado

- Se define la zona horaria de referencia `MADRID_TZ` (Europa/Madrid), coherente con el contexto del presente trabajo.
- Se implementan funciones auxiliares (_helpers_) para limpiar HTML, normalizar textos y manipular campos temporales,
que se reutilizan en las funciones de obtención de noticias.


In [ ]:
import requests
import feedparser
from urllib.parse import quote
from datetime import datetime, timezone
from zoneinfo import ZoneInfo
import re

MADRID_TZ = ZoneInfo("Europe/Madrid")


# ---------------------------
# Helpers
# ---------------------------
def _clean_html(text: str) -> str:
    if not text:
        return ""
    return re.sub(r"<[^>]+>", "", text).strip()


def _today_local(tz=MADRID_TZ) -> datetime.date:
    return datetime.now(tz).date()


def _parse_gdelt_seendate(seendate: str):
    """
    Soporta ambos formatos que puede devolver GDELT:
    - 'YYYYMMDDHHMMSS' (antiguo)
    - 'YYYYMMDDTHHMMSSZ' o 'YYYYMMDDT HHMMSS Z' (ej: 20260106T073000Z)
    Devuelve datetime en UTC con tzinfo.
    """
    if not seendate:
        return None

    s = seendate.strip()

    # Formato: 20260106T073000Z
    try:
        if "T" in s and s.endswith("Z"):
            dt_utc = datetime.strptime(s, "%Y%m%dT%H%M%SZ").replace(tzinfo=timezone.utc)
            return dt_utc
    except ValueError:
        pass

    # Formato: 20260106073000
    try:
        dt_utc = datetime.strptime(s, "%Y%m%d%H%M%S").replace(tzinfo=timezone.utc)
        return dt_utc
    except ValueError:
        return None






### Cliente RSS de GDELT: extracción de noticias recientes

Esta celda implementa la lógica para consumir el endpoint de documentos de **GDELT** y obtener noticias recientes relacionadas con una consulta (`query`):

- Construye la URL a la API de GDELT (`/api/v2/doc/doc`).
- Ajusta la ventana temporal de interés (`hours`), típica de 24–48 h, para capturar noticias recientes.
- Aplica funciones de limpieza y normalización de fechas a la salida.

Esta funcionalidad proporciona una de las dos patas del enfoque híbrido de obtención de noticias (GDELT + Google News RSS) utilizado en el TFM.


In [ ]:
# ---------------------------
# GDELT


from datetime import timedelta

def fetch_gdelt_last_hours(query: str, n: int = 3, hours: int = 48, sourcelang: str = "eng", tz=MADRID_TZ):
    url = "https://api.gdeltproject.org/api/v2/doc/doc"

    # Sanitizar query para GDELT
    query = query.replace("&", " and ")

    now_utc = datetime.now(timezone.utc)
    start_utc = now_utc - timedelta(hours=hours)

    params = {
        "query": query,
        "mode": "ArtList",
        "format": "JSON",
        "maxrecords": 250,
        "sort": "datedesc",
        "startdatetime": start_utc.strftime("%Y%m%d%H%M%S"),
        "enddatetime": now_utc.strftime("%Y%m%d%H%M%S"),
    }
    if sourcelang:
        params["sourcelang"] = sourcelang

    headers = {"User-Agent": "Mozilla/5.0", "Accept": "application/json,*/*"}

    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()

    ct = (r.headers.get("content-type") or "").lower()
    if "json" not in ct:
        raise RuntimeError(f"GDELT no devolvió JSON (ct={ct}): {r.text[:200]!r}")

    data = r.json()

    results = []
    seen_titles = set()

    for art in data.get("articles", []) or []:
        title = (art.get("title") or "").strip()
        link = (art.get("url") or "").strip()
        domain = (art.get("domain") or "").lower().strip()
        seendate = art.get("seendate")

        if not title or not link:
            continue

        # dedup por título
        tkey = title.lower()
        if tkey in seen_titles:
            continue
        seen_titles.add(tkey)

        # Parse fecha
        dt_utc = _parse_gdelt_seendate(seendate)
        published_utc = dt_utc.isoformat() if dt_utc else None
        published_local = dt_utc.astimezone(tz).isoformat() if dt_utc else None

        results.append({
            "provider": "gdelt",
            "title": title,
            "link": link,
            "source": art.get("source"),
            "domain": domain,
            "seendate": seendate,
            "published_utc": published_utc,
            "published_local": published_local,
            "summary": art.get("summary") or None,
        })

        if len(results) >= n:
            break

    return results




### Cliente de Google News RSS (fallback)

En esta celda se construye un **mecanismo de respaldo** basado en Google News RSS:

- Se genera una URL RSS parametrizada con:
  - `query`: término o conjunto de términos de búsqueda.
  - `lang` y `country`: idioma y país de referencia.
- Se aplican filtros temporales mediante `max_age_days` para limitar las noticias a una ventana reciente (por defecto, últimos 7 días).
- Se parsea el feed con `feedparser` y se normalizan campos como título, resumen, enlace, fecha de publicación y fuente.

Este cliente se utiliza como complemento a GDELT para obtener una mayor cobertura  de noticias.


In [ ]:
# ---------------------------
# Google News RSS (fallback)


from email.utils import parsedate_to_datetime
from datetime import timedelta

def fetch_google_rss(
    query: str,
    n: int = 3,
    lang: str = "en",
    country: str = "US",
    tz=MADRID_TZ,
    max_age_days: int = 7,   # <-- filtro: últimos 7 días
):
    url = (
        f"https://news.google.com/rss/search?"
        f"q={quote(query)}"
        f"&hl={lang}"
        f"&gl={country}"
        f"&ceid={country}:{lang}"
    )
    feed = feedparser.parse(url)

    results = []
    seen = set()

    now_utc = datetime.now(timezone.utc)
    cutoff_utc = now_utc - timedelta(days=max_age_days)

    for e in feed.entries:
        title = (getattr(e, "title", "") or "").strip()
        link = (getattr(e, "link", "") or "").strip()
        if not title or not link:
            continue

        key = title.lower()
        if key in seen:
            continue
        seen.add(key)

        summary = _clean_html(getattr(e, "summary", "") or "") or None
        published_raw = getattr(e, "published", None)

        # Parse fecha RSS → UTC
        dt_utc = None
        if published_raw:
            try:
                dt = parsedate_to_datetime(published_raw)
                if dt.tzinfo is None:
                    dt = dt.replace(tzinfo=timezone.utc)
                dt_utc = dt.astimezone(timezone.utc)
            except Exception:
                dt_utc = None

        # Filtrar por recencia (si no hay fecha, lo descartamos para evitarregistros que son antiguos)
        if dt_utc is None:
            continue
        if dt_utc < cutoff_utc:
            continue

        results.append({
            "provider": "rss",
            "title": title,
            "link": link,
            "source": getattr(getattr(e, "source", None), "title", None),
            "published": published_raw,
            "published_utc": dt_utc.isoformat(),
            "published_local": dt_utc.astimezone(tz).isoformat(),
            "summary": summary,
        })

        if len(results) >= n:
            break

    return results

### Construcción de queries semánticas para índices bursátiles y activos

La función `build_query` transforma una entrada simple del usuario (por ejemplo, `SP500`, `NASDAQ100`, `IBEX35`) en una **query expandida** apta para:

- GDELT (parámetros de búsqueda avanzados).
- Feeds RSS (Google News, agregadores financieros).

Características clave:

- Mapea alias conocidos (SP500, S&P500, NASDAQ100, IBEX35, etc.) a expresiones con sinónimos, tickers y ETFs representativos.
- Permite capturar distintas formas en las que la prensa financiera se refiere al mismo índice.
- Devuelve una cadena lista para ser utilizada en las funciones de consulta posteriores.

Esta capa de aliasing mejora la **recuperación de noticias relevantes** para el activo/índice de interés en el marco del TFM.


In [ ]:
def load_aliases_from_txt(path: str) -> dict[str, str]:
    aliases = {}
    with open(path, encoding="utf-8") as f:
        for line in f:
            if "=" not in line:
                continue
            key, value = line.split("=", 1)
            aliases[key.strip().upper()] = value.strip()
    return aliases


In [ ]:
ALIASES = load_aliases_from_txt("C:\\Users\\migue\\Documents\\TFM\\aliases.txt")

def build_query(user_input: str) -> str:
    s = user_input.strip()
    key = s.upper().replace(" ", "")
    if key in ALIASES:
        return ALIASES[key]

    if s.isupper() and len(s) <= 6:
        return f'("{s}" AND (stock OR shares OR earnings OR market))'

    if " " in s:
        return f'("{s}" AND (stock OR shares OR earnings OR market))'

    return s


In [ ]:
def build_query_old(user_input: str) -> str:
    """
    Convierte una entrada simple del usuario en una query usable para GDELT/RSS.
    """
    s = user_input.strip()

    aliases = {
        "SP500": '"S and P 500" OR "S&P500" OR SPX OR "^GSPC" OR "SPY ETF" OR "SPDR S&P 500 ETF" OR "SPDR S&P 500 ETF Trust"',
        "S&P500": '"S and P 500" OR "S&P500" OR SPX OR "^GSPC" OR "SPY ETF" OR "SPDR S&P 500 ETF" OR "SPDR S&P 500 ETF Trust"',
        "NASDAQ100": '("Nasdaq 100" OR NDX OR QQQ)',
        "IBEX35": '("IBEX 35" OR IBEX)',
    }

    key = s.upper().replace(" ", "")
    if key in aliases:
        return aliases[key]

    # Si parece ticker (AAPL, TSLA, SAN, etc.)
    if s.isupper() and len(s) <= 6:
        return f'("{s}" AND (stock OR shares OR earnings OR market))'

    # Si es texto libre (nombre empresa)
    if " " in s:
        return f'("{s}" AND (stock OR shares OR earnings OR market))'

    return s


### Construcción de URLs para feeds RSS (query final)

La función `make_rss_query_from_base` recibe una URL base de RSS (por ejemplo, de Google News) y una `query` ya procesada:

- Ensambla la URL completa con parámetros codificados correctamente.
- Garantiza que los operadores lógicos y caracteres especiales se transmiten en un formato aceptado por el proveedor RSS.

El resultado es una URL lista para ser consumida por `feedparser`, integrando así la capa de construcción de consultas con la capa de obtención de datos.


In [ ]:
def make_rss_query_from_base(base: str) -> str:
    """
    Convierte una base tipo GDELT (ORs) en una query simple para Google News RSS.
    """
    q = base
    q = re.sub(r"\bsourcelang:\w+\b", "", q, flags=re.IGNORECASE)
    q = q.replace("(", " ").replace(")", " ")
    q = re.sub(r"\s+", " ", q).strip()
    return q


def fetch_news_hybrid_3_and_3(
    user_input: str,
    gdelt_n: int = 3,
    rss_n: int = 3,
    gdelt_lang: str = "eng",
    rss_lang: str = "en",
    rss_country: str = "US",
    hours: int = 48,
    rss_max_age_days: int = 2,   # 👈 AQUÍ controlas ayer + hoy
):
    """
    Devuelve (si existen):
    - 3 artículos de GDELT (últimas `hours` horas)
    - 3 artículos de RSS (últimos `rss_max_age_days` días)

    No hace fallback cruzado: cada fuente es independiente.
    """

    # --------------------------------------------------
    # Construir queries
    # --------------------------------------------------
    base = build_query(user_input)  # OR plano

    query_gdelt = f'sourcelang:english ({base}) AND market'
    query_rss = make_rss_query_from_base(base)

    # --------------------------------------------------
    # 1) GDELT
    # --------------------------------------------------
    gdelt_items = []
    try:
        gdelt_items = fetch_gdelt_last_hours(
            query_gdelt,
            n=gdelt_n,
            hours=hours,
            sourcelang=gdelt_lang
        )
    except Exception as e:
        print("GDELT error:", e)
        gdelt_items = []

    for it in gdelt_items:
        it.setdefault("provider", "gdelt")

    # --------------------------------------------------
    # 2) RSS (con filtro de días)
    # --------------------------------------------------
    rss_items = []
    try:
        rss_items = fetch_google_rss(
            query_rss,
            n=rss_n,
            lang=rss_lang,
            country=rss_country,
            max_age_days=rss_max_age_days
        )
    except Exception as e:
        print("RSS error:", e)
        rss_items = []

    for it in rss_items:
        it.setdefault("provider", "rss")

    # --------------------------------------------------
    # 3) Merge + dedup global
    # --------------------------------------------------
    merged = []
    seen_links = set()
    seen_titles = set()

    def add_item(item):
        link = (item.get("link") or "").strip()
        title = (item.get("title") or "").strip().lower()

        if link and link in seen_links:
            return
        if title and title in seen_titles:
            return

        if link:
            seen_links.add(link)
        if title:
            seen_titles.add(title)

        merged.append(item)

    for it in gdelt_items[:gdelt_n]:
        add_item(it)

    for it in rss_items[:rss_n]:
        add_item(it)

    return {
        "source": "gdelt+rss",
        "queries": {
            "gdelt": query_gdelt,
            "rss": query_rss,
        },
        "counts": {
            "gdelt_requested": gdelt_n,
            "rss_requested": rss_n,
            "gdelt_returned": len(gdelt_items[:gdelt_n]),
            "rss_returned": len(rss_items[:rss_n]),
            "total_returned": len(merged),
        },
        "items": merged,
    }


### Ejecución de la metodología implementada híbrida de noticias (GDELT + RSS) para losa ctivos definidos.

En esta celda se invoca `fetch_news_hybrid_3_and_3` con el término `"SP500"` y una restricción temporal (`rss_max_age_days=2`) que acota las noticias a ayer + hoy.

Objetivos de la celda:

- Comprobar que el pipeline de obtención de noticias funciona de extremo a extremo.
- Inspeccionar el número de noticias devueltas por cada proveedor (`counts`).
- Revisar títulos, fechas y fuentes para validar la calidad de los datos que se utilizarán más adelante en el análisis de sentimiento.

Esta celda es útil a modo de **prueba de integración** dentro del TFM.


### Aqui yo realizo las pruebas para probar que el diccionario funciona con los activos.

In [ ]:
out = fetch_news_hybrid_3_and_3(
    "IBEX35",
    rss_max_age_days=2
)

print(out["counts"])
for i, item in enumerate(out["items"], 1):
    print(f"\n{i}. [{item['provider'].upper()}] {item['title']}")
    print("   Date:", item.get("published_local") or item.get("published_utc") or item.get("published"))
    print("   Source:", item.get("source") or item.get("domain"))


{'gdelt_requested': 3, 'rss_requested': 3, 'gdelt_returned': 1, 'rss_returned': 3, 'total_returned': 4}

1. [GDELT] Dollar sold ; JPY gains amid rate check speculation - Newsquawk US Market Wrap
   Date: 2026-01-24T01:45:00+01:00
   Source: zerohedge.com

2. [RSS] Spain stocks lower at close of trade; IBEX 35 down 0.67% - Investing.com
   Date: 2026-01-23T18:30:11+01:00
   Source: Investing.com

3. [RSS] Spain shares lower at close of trade; IBEX 35 down 0.67% - Investing.com India
   Date: 2026-01-23T18:27:43+01:00
   Source: Investing.com India

4. [RSS] Spain shares lower at close of trade; IBEX 35 down 0.67% - Investing.com UK
   Date: 2026-01-23T18:05:00+01:00
   Source: Investing.com UK


### Inspección detallada de los ítems devueltos por el pipeline híbrido

Partiendo de la salida `out["items"]` generada en la celda anterior, aquí se recorre la lista de noticias para:

- Imprimir título, enlace, fuente y fecha normalizada de cada ítem.
- Verificar que la información relevante para el análisis (título, resumen, metadatos temporales) esté correctamente formateada.

In [ ]:
for i, item in enumerate(out["items"], 1):
    print(f"\n{i}. [{item['provider'].upper()}] {item['title']}")
    print("   Link:", item.get("link"))
    print("   Source:", item.get("source") or item.get("domain"))
    date = item.get("published_local") or item.get("published_utc") or item.get("published")
    print("   Date:", date)
    if item.get("summary"):
        print("   Summary:", item["summary"])



1. [GDELT] Dollar sold ; JPY gains amid rate check speculation - Newsquawk US Market Wrap
   Link: https://www.zerohedge.com/markets/dollar-sold-jpy-gains-amid-rate-check-speculation-newsquawk-us-market-wrap
   Source: zerohedge.com
   Date: 2026-01-24T01:45:00+01:00

2. [RSS] Spain stocks lower at close of trade; IBEX 35 down 0.67% - Investing.com
   Link: https://news.google.com/rss/articles/CBMirwFBVV95cUxNSzJDakNhSWlHR2dpdkVsZmozRzFCVk5YN19XVkVBeVFfYm45Uk9hSzI0UDNURE9SUlJoWlc3WnFKQ0FJc0JhTDVSbVBCdkhyR3FoeVlYS1hSRXlMd2ZNN0VBbDJaZ2JqUGtXVzFzYkNBSkMwTkN0RFNBN24tVXRKVEg1aENlNFpGekFwaWxLWE1kLTFIRUJBb2NrXzktek8xdnRLVjhReUc1TC16ZE5z?oc=5
   Source: Investing.com
   Date: 2026-01-23T18:30:11+01:00
   Summary: Spain stocks lower at close of trade; IBEX 35 down 0.67%&nbsp;&nbsp;Investing.com

3. [RSS] Spain shares lower at close of trade; IBEX 35 down 0.67% - Investing.com India
   Link: https://news.google.com/rss/articles/CBMirgFBVV95cUxQWWRTVGdOeGxac21LekZsbm1tYXBlU3pwSDFZamVfNHEzU2NzTFV

In [ ]:
#!pip install -q transformers torch

### Carga del modelo de sentimiento financiero (FinBERT)

Aquí se inicializa un `pipeline` de `transformers` para la tarea de `sentiment-analysis` utilizando el modelo:

- `ProsusAI/finbert`, una variante de BERT especializada en **texto financiero**.

Este componente es clave en el TFM, ya que convierte textos de noticias en etiquetas de sentimiento (`positive`, `neutral`, `negative`) con una probabilidad asociada, lo que más adelante se traducirá en una señal numérica.


In [ ]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="ProsusAI/finbert",
    tokenizer="ProsusAI/finbert"
)

C:\Users\migue\anaconda3\envs\nlp-news\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


### Comprobación de versiones de `torch` y `transformers`

Esta celda imprime las versiones instaladas de:

- `torch`
- `transformers`

Para este caso se tiene que tener una versión de torch arriba de 2.6

In [ ]:
import torch, transformers
print(torch.__version__)
print(transformers.__version__)

2.9.1+cpu
4.57.6


### Construcción del texto de entrada para el modelo de sentimiento

La función `build_text_for_sentiment` define cómo se combina la información de cada noticia para alimentar el modelo de análisis de sentimiento:

- Prioriza el uso conjunto de `title` + `summary` cuando ambos están disponibles.
- Si solo hay título, utiliza únicamente ese campo.

Esta decisión controla el **contexto textual** con el que FinBERT evalúa cada noticia y, por tanto, influye en la clasificación de sentimiento que se obtiene.


In [ ]:
def build_text_for_sentiment(item: dict) -> str:
    title = (item.get("title") or "").strip()
    summary = (item.get("summary") or "").strip()
    if summary:
        return f"{title}. {summary}"
    return title

### Ejecución del pipeline de noticias para el activo elegido (entrada al análisis de sentimiento)

En esta celda se vuelve a invocar `fetch_news_hybrid_3_and_3`, en este caso de forma más directa, y se almacena el resultado en:

- `out`: diccionario con metadatos y la lista de noticias.
- `items`: lista de ítems, cada uno con título, enlace, fechas y, eventualmente, resumen.

Estos `items` son la **materia prima** sobre la que se aplicará el modelo de sentimiento en las celdas siguientes.


### En la siguiente celda, la entrada es la clave para el activo.

In [ ]:
out = fetch_news_hybrid_3_and_3("SP500")
items = out["items"]

In [ ]:
out = fetch_news_hybrid_3_and_3("IBEX35")
items = out["items"]

### Aplicación del modelo de sentimiento a cada noticia

Esta celda recorre la lista `items` y, para cada noticia:

1. Construye el texto de entrada mediante `build_text_for_sentiment`.
2. Llama al `sentiment_model` (FinBERT financiero) para obtener:
   - `label`: `positive`, `neutral` o `negative`.
   - `score`: confianza asociada a esa predicción.
3. Almacena en el propio diccionario de la noticia:
   - `sentiment` (etiqueta categórica).
   - `sentiment_score` (probabilidad).

De este modo, cada noticia pasa a estar enriquecida con información de sentimiento.


In [ ]:
for item in items:
    text = build_text_for_sentiment(item)
    if not text:
        item["sentiment"] = None
        item["sentiment_score"] = None
        continue

    pred = sentiment_model(text)[0]   # {'label': 'positive'/'neutral'/'negative', 'score': ...}

    item["sentiment"] = pred["label"]
    item["sentiment_score"] = float(pred["score"])


### Transformación del sentimiento categórico a valor numérico

En esta celda se define el mapeo:

- `positive` → `+1`
- `neutral`  → `0`
- `negative` → `−1`

y se calcula, para cada noticia, el campo:

- `sentiment_value = LABEL_TO_NUM[label] * score`

Es decir, el signo viene dado por la polaridad y la magnitud por la confianza del modelo. Este valor escalar es especialmente útil para la construcción de indicadores cuantitativos de sentimiento en el marco del TFM.


In [ ]:
for i, item in enumerate(items, 1):
    date = item.get("published_local") or item.get("published_utc") or item.get("published")
    print(f"\n{i}. [{item['provider'].upper()}] {item['title']}")
    print("   Date:", date)
    print("   Sentiment:", item.get("sentiment"), f"({item.get('sentiment_score'):.2f})" if item.get("sentiment_score") is not None else "")
    print("   Link:", item.get("link"))



1. [GDELT] Dollar sold ; JPY gains amid rate check speculation - Newsquawk US Market Wrap
   Date: 2026-01-24T01:45:00+01:00
   Sentiment: positive (0.83)
   Link: https://www.zerohedge.com/markets/dollar-sold-jpy-gains-amid-rate-check-speculation-newsquawk-us-market-wrap

2. [RSS] Spain stocks lower at close of trade; IBEX 35 down 0.67% - Investing.com
   Date: 2026-01-23T18:30:11+01:00
   Sentiment: negative (0.97)
   Link: https://news.google.com/rss/articles/CBMirwFBVV95cUxNSzJDakNhSWlHR2dpdkVsZmozRzFCVk5YN19XVkVBeVFfYm45Uk9hSzI0UDNURE9SUlJoWlc3WnFKQ0FJc0JhTDVSbVBCdkhyR3FoeVlYS1hSRXlMd2ZNN0VBbDJaZ2JqUGtXVzFzYkNBSkMwTkN0RFNBN24tVXRKVEg1aENlNFpGekFwaWxLWE1kLTFIRUJBb2NrXzktek8xdnRLVjhReUc1TC16ZE5z?oc=5

3. [RSS] Spain shares lower at close of trade; IBEX 35 down 0.67% - Investing.com India
   Date: 2026-01-23T18:27:43+01:00
   Sentiment: negative (0.97)
   Link: https://news.google.com/rss/articles/CBMirgFBVV95cUxQWWRTVGdOeGxac21LekZsbm1tYXBlU3pwSDFZamVfNHEzU2NzTFVsbVdaMUVqc0E3RlpSV3

### Inspección de resultados de sentimiento a nivel de noticia

A partir de los valores calculados en la celda anterior, aquí se imprime, para cada noticia:

- Título.
- Etiqueta de sentimiento (`sentiment`) y su probabilidad (`sentiment_score`).
- Valor numérico agregado (`sentiment_value`).

Esta inspección permite comprobar la coherencia de las predicciones del modelo de sentimiento con el contenido intuitivo de las noticias, añadiendo una capa de validación cualitativa al TFM.


In [ ]:
LABEL_TO_NUM = {"positive": 1, "neutral": 0, "negative": -1}

for item in items:
    label = item.get("sentiment")
    score = item.get("sentiment_score")
    if label is None or score is None:
        item["sentiment_value"] = None
    else:
        item["sentiment_value"] = LABEL_TO_NUM[label] * score

for i, it in enumerate(items, 1):
    print(f"{i}. {it['title']}")
    print("   Sentiment:", it.get("sentiment"), f"({it.get('sentiment_score'):.2f})")
    print("   Sentiment value:", it.get("sentiment_value"))


1. Dollar sold ; JPY gains amid rate check speculation - Newsquawk US Market Wrap
   Sentiment: positive (0.83)
   Sentiment value: 0.8326207399368286
2. Spain stocks lower at close of trade; IBEX 35 down 0.67% - Investing.com
   Sentiment: negative (0.97)
   Sentiment value: -0.9684360027313232
3. Spain shares lower at close of trade; IBEX 35 down 0.67% - Investing.com India
   Sentiment: negative (0.97)
   Sentiment value: -0.9703910946846008
4. Spain shares lower at close of trade; IBEX 35 down 0.67% - Investing.com UK
   Sentiment: negative (0.97)
   Sentiment value: -0.9708607792854309


### Agregación diaria del sentimiento de mercado

En esta celda se construye un `DataFrame` de `pandas` con la información de sentimiento y se realiza una agregación temporal:

1. Se seleccionan columnas clave: `published_local`, `title`, `sentiment`, `sentiment_score`, `sentiment_value`.
2. Se extrae la fecha (sin hora) en una nueva columna `date`.
3. Se calcula, para cada día, la **media del `sentiment_value`**.

El resultado es una serie temporal diaria de sentimiento.


In [ ]:
values = [it.get("sentiment_value") for it in items]
#values
import pandas as pd

df = pd.DataFrame(items)
df_small = df[["published_local", "title", "sentiment", "sentiment_score", "sentiment_value"]].copy()
df_small["date"] = pd.to_datetime(df_small["published_local"]).dt.date

daily = df_small.groupby("date")["sentiment_value"].mean().reset_index()
print(daily)


         date  sentiment_value
0  2026-01-23        -0.969896
1  2026-01-24         0.832621
